In [95]:
import pandas as pd
import os
import json
from datetime import datetime

In [96]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# This code automatically detects if it's running inside our Docker container
if os.path.isdir('/app'):
    # If the '/app' directory exists, we are in Docker.
    print("Running inside Docker container. Using path: ./data")
    BASE_DATA_PATH = './data'
else:
    # Otherwise, we are running locally.
    print("Running locally. Using path: ../data")
    BASE_DATA_PATH = r'C:\Users\ben\Documents\Programming\Python\Python use sendiri\Test_InovasiDS\Data Engineer Test [NEW] (2)\dwh-coding-challenge\data'

#error handling 
if not os.path.isdir(BASE_DATA_PATH):
    print(f"ERROR: The path '{BASE_DATA_PATH}' does not exist")

Running locally. Using path: ../data


In [97]:
def load_events(table_name: str) -> list:
    events = []
    directory_path = os.path.join(BASE_DATA_PATH, table_name)

    # Check if the directory for the table exists
    if not os.path.isdir(directory_path):
        print(f"Warning: Directory not found for table '{table_name}' at {directory_path}")
        return events

    # Loop through every file in the directory
    for filename in os.listdir(directory_path):
        # Process only the files that end with .json
        if filename.endswith('.json'):
            file_path = os.path.join(directory_path, filename)
            with open(file_path, 'r') as f:
                # Each file contains a single JSON object, so use json.load()
                event_data = json.load(f)
                events.append(event_data)

    # This ensures they are processed in the correct chronological order.
    events.sort(key=lambda x: x['ts'])
    return events


In [98]:
def reconstruct_table(events: list) -> pd.DataFrame:
    records = {}
    for event in events:
        record_id = event['id']
        op = event['op']

        if op == 'c':
            records[record_id] = event['data']
            records[record_id]['ts'] = event['ts']
        elif op == 'u':
            if record_id in records:
                records[record_id].update(event['set'])
                records[record_id]['ts'] = event['ts']

    df = pd.DataFrame.from_dict(records, orient='index')
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'id'}, inplace=True)
    return df


In [99]:
def display_formatted_df(df, title):
    print(f"\n--- {title} ---")
    
    if df.empty:
        print("Table is empty.")
        return

    # Create a copy
    styled_df = df.copy()

    # Convert any column ending with 'ts' from milliseconds to datetime strings
    for col in styled_df.columns:
        if col.endswith('ts'):
            # pd.to_numeric is important in case some values are missing (NaN)
            styled_df[col] = pd.to_numeric(styled_df[col], errors='coerce')
            styled_df[col] = pd.to_datetime(styled_df[col], unit='ms').dt.strftime('%Y-%m-%d %H:%M:%S')

    # Replace any missing values for a cleaner look in the terminal
    if os.path.isdir('/app'): #display for docker terminal
        styled_df.fillna('N/A', inplace=True)
        print(styled_df.to_string(index=False))
    else: #display for local jupyter notebook
        styler = styled_df.style.set_caption(f"<h3>{title}</h3>").hide(axis='index').format(
            {'credit_used': '{:,.0f}', 'monthly_limit': '{:,.0f}'},
            na_rep="N/A"
        )
        display(styler)

In [100]:
account_events = load_events('accounts')
card_events = load_events('cards')
saving_account_events = load_events('saving_accounts')

In [101]:
# --- TASK 1: Visualize Historical Tables (IMPROVED) ---
print("## TASK 1: VISUALIZE HISTORICAL TABLES ##")
accounts_df = reconstruct_table(account_events)
cards_df = reconstruct_table(card_events)
savings_df = reconstruct_table(saving_account_events)

display_formatted_df(accounts_df, "Historical Accounts Table")
display_formatted_df(cards_df, "Historical Cards Table")
display_formatted_df(savings_df, "Historical Savings Accounts Table")


## TASK 1: VISUALIZE HISTORICAL TABLES ##

--- Historical Accounts Table ---


id,account_id,name,address,phone_number,email,ts,savings_account_id,card_id
a1globalid,a1,Anthony,Jakarta,87654321,anthony@anotherbank.com,2020-01-16 08:30:00,sa1,c2



--- Historical Cards Table ---


id,card_id,card_number,credit_used,monthly_limit,status,ts
c1globalid,c1,11112222,0,"30,000",CLOSED,2020-01-15 09:00:00
c2globalid,c2,12123434,"37,000","70,000",ACTIVE,2020-01-18 15:30:00



--- Historical Savings Accounts Table ---
Table is empty.


In [102]:
# --- TASK 2: Visualize Denormalized Joined Table (IMPROVED) ---
print("\n\n## TASK 2: VISUALIZE DENORMALIZED JOINED TABLE ##")
denormalized_df = pd.merge(
    accounts_df,
    cards_df,
    on='card_id',
    how='left',
    suffixes=('_account', '_card')
)
denormalized_df = pd.merge(
    denormalized_df,
    savings_df,
    left_on='savings_account_id',
    right_on='id',
    how='left',
    suffixes=('', '_savings')
)

# Reorder columns for logical grouping and readability
try:
    final_columns_order = [
        # Account Info
        'id_account', 'account_id', 'name', 'email', 'phone_number', 'address', 'ts_account',
        # Card Info
        'card_id', 'card_number', 'credit_used', 'monthly_limit', 'status', 'ts_card',
        # Savings Info
        'savings_account_id', 'id_savings', 'ts_savings'
    ]
    # Filter the list to only include columns that actually exist in the DataFrame
    existing_columns = [col for col in final_columns_order if col in denormalized_df.columns]
    denormalized_df = denormalized_df[existing_columns]
except KeyError:
    print("Could not reorder columns, some expected columns were not found.")

display_formatted_df(denormalized_df, "Complete Denormalized Table")





## TASK 2: VISUALIZE DENORMALIZED JOINED TABLE ##

--- Complete Denormalized Table ---


id_account,account_id,name,email,phone_number,address,ts_account,card_id,card_number,credit_used,monthly_limit,status,ts_card,savings_account_id
a1globalid,a1,Anthony,anthony@anotherbank.com,87654321,Jakarta,1579163400000,c2,12123434,"37,000","70,000",ACTIVE,1579361400000,sa1


In [103]:
# --- TASK 3: Transaction Analysis (No changes needed) ---

def find_transactions(table_name: str, key_field: str, events: list):
    print(f"\n--- Analyzing Transactions for '{table_name}' ---")
    transaction_count = 0
    for event in events:
        if event['op'] == 'u' and key_field in event['set']:
            transaction_count += 1
            record_id = event['id']
            timestamp = datetime.fromtimestamp(event['ts'] / 1000).strftime('%Y-%m-%d %H:%M:%S')
            value = event['set'][key_field]
            print(f"  - Transaction detected on '{table_name}' (ID: {record_id})")
            print(f"    - When: {timestamp}")
            print(f"    - How much: The '{key_field}' was updated to {value}")
    if transaction_count == 0:
        print("  No transaction activities found.")
    else:
        print(f"\nTotal transactions found in '{table_name}': {transaction_count}")

print("\n\n## TASK 3: TRANSACTION ANALYSIS ##")
print("Transaction is defined as an activity which changes the balance of the savings account or credit used of the card.")
find_transactions('saving_accounts', 'balance', saving_account_events)
find_transactions('cards', 'credit_used', card_events)



## TASK 3: TRANSACTION ANALYSIS ##
Transaction is defined as an activity which changes the balance of the savings account or credit used of the card.

--- Analyzing Transactions for 'saving_accounts' ---
  No transaction activities found.

--- Analyzing Transactions for 'cards' ---
  - Transaction detected on 'cards' (ID: c1globalid)
    - When: 2020-01-06 19:30:00
    - How much: The 'credit_used' was updated to 12000
  - Transaction detected on 'cards' (ID: c1globalid)
    - When: 2020-01-08 01:00:00
    - How much: The 'credit_used' was updated to 19000
  - Transaction detected on 'cards' (ID: c1globalid)
    - When: 2020-01-10 18:00:00
    - How much: The 'credit_used' was updated to 0
  - Transaction detected on 'cards' (ID: c2globalid)
    - When: 2020-01-18 22:30:00
    - How much: The 'credit_used' was updated to 37000

Total transactions found in 'cards': 4
